<h1 align="center">Custom Vision - Cognitive services Lab 1</h1>
<h1 align="center">Image Classification</h1>
<h1 align="center">Meetup DFW Data & AI - Microsoft</h1>
## Setting Up Environment

1. Go to https://customvision.ai/  and sign in with your azure account/ID
2. Once you are in, go to Settings (gear icon - upper right corner) and copy your Limited Trial Prediction and Training Key

In [1]:
import sys

In [ ]:
#This is only one time run to install the required python libraries on this virtual machine
!{sys.executable} -m pip install -U azure-cognitiveservices-vision-customvision

## Initialize the libraries and origin and destination workspaces

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com"

# DESTINATION Resource Group Keys
dest_training_key = "70b0304d1e69481693e7d5e249212adf"
dest_prediction_key = "7e1dcf4b76114b96a4e00e728a8cc6d7"
dest_project_new_name = "DebrisRecognition_Copy"

# ORIGIN Resource Group Keys
training_key = "70b0304d1e69481693e7d5e249212adf"
prediction_key = "7e1dcf4b76114b96a4e00e728a8cc6d7"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)
dest_trainer = CustomVisionTrainingClient(dest_training_key, endpoint=ENDPOINT)

# Find the image classification domain
classification_domain = next(domain for domain in trainer.get_domains() if domain.type == "Classification")
dest_classification_domain = next(domain for domain in dest_trainer.get_domains() if domain.type == "Classification")

## Get the origin project ID reference

In [3]:
myProjects = trainer.get_projects()

In [4]:
for project in myProjects:
    print(project.name)
    print(project.id)
    print(project.description)

DebrisRecognition
aadaa47b-ca7b-4536-8450-ec4428dd302e



In [5]:
Project = trainer.get_project(project_id="aadaa47b-ca7b-4536-8450-ec4428dd302e")

## Create the destination Project 

In [6]:
dest_Project = dest_trainer.create_project(dest_project_new_name, domain_id=dest_classification_domain.id)

In [7]:
for project in dest_trainer.get_projects():
    print(project.name)
    print(project.id)
    print(project.description)

DebrisRecognition_Copy
76a87d7b-dde8-437a-b59d-6bcf17145dcf

DebrisRecognition
aadaa47b-ca7b-4536-8450-ec4428dd302e



## Get the tags on origin project and create same tags on destination project

In [8]:
dest_tags = []
for tag in trainer.get_tags(Project.id):
    dest_tags.append(dest_trainer.create_tag(dest_Project.id, tag.name))
    print(tag.name)

Ocean
Object


In [9]:
dest_tags_dict = {}
dest_tag_ids = []
for tag in dest_tags:
    dest_tags_dict[tag.name] = tag.id
    dest_tag_ids.append(tag.id)

print(dest_tags_dict)
print(dest_tag_ids)

{'Ocean': 'f8df763d-8f30-4d7c-b901-a62e70293194', 'Object': '025f5966-f499-4d43-83b2-ddd6c76a0bfd'}
['f8df763d-8f30-4d7c-b901-a62e70293194', '025f5966-f499-4d43-83b2-ddd6c76a0bfd']


## Get the images on origin project

In [10]:
tagged_images = trainer.get_tagged_images(Project.id, take=trainer.get_tagged_image_count(Project.id))

In [12]:
tagged_images_with_tags = []
for image in tagged_images: #for each tagged image on origin
    dest_tags_ids = []
    
    for tag in image.tags: #for each tag on the origin image
        dest_tags_ids.append(dest_tags_dict[tag.tag_name]) #append it to the image dest_tags_ids list
    
    tagged_images_with_tags.append(ImageUrlCreateEntry(url=image.original_image_uri, tag_ids=dest_tags_ids))
print("Done")

Done


## Create the images with regions on destination project

In [13]:
limit = 64 # this is a limit imposed on the API, so we need to batch the creation process
count_of_images = len(tagged_images_with_tags)

for i in range(0,count_of_images,limit):
    begin=i
    end=limit+i
    if(end > count_of_images ): end = count_of_images
    dest_trainer.create_images_from_urls(dest_Project.id, images=tagged_images_with_tags[begin:end])

In [14]:
print("Count of Tagged images on origin project: " + str(trainer.get_tagged_image_count(Project.id)))
print("Count of Tagged images on destination project: " + str(dest_trainer.get_tagged_image_count(dest_Project.id)))

Count of Tagged images on origin project: 102
Count of Tagged images on destination project: 102


## Train the model on destination Project

In [15]:
import time

print ("Training... (showing status each 10 seconds)")
iteration = dest_trainer.train_project(dest_Project.id)
while (iteration.status != "Completed"):
    iteration = dest_trainer.get_iteration(dest_Project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

# The iteration is now trained. Make it the default project endpoint
dest_trainer.update_iteration(dest_Project.id, iteration.id, is_default=True)
print ("Done!")

Training... (showing status each 10 seconds)
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


## Get performance metrics with a custom threshold

In [16]:
performance = dest_trainer.get_iteration_performance(dest_Project.id, iteration.id, threshold=0.80)
print("Iteration: " + iteration.name)
print("Precision: " + str(performance.precision * 100)+ "%")
print("Recall: " + str(performance.recall * 100) + "%")

Iteration: Iteration 1
Precision: 91.85606%
Recall: 88.25821%


In [17]:
per_tag_performance = performance.per_tag_performance

In [18]:
for tag in per_tag_performance:
    print ("Tag: " + tag.name + "\t\t" + "Precision: " + str(tag.precision) + "\t" + "Recall: " + str(tag.recall))

Tag: Object		Precision: 0.949735463	Recall: 0.872222245
Tag: Ocean		Precision: 0.8888889	Recall: 0.8992674


## Get and use the default prediction endpoint

In [19]:
#You can make a web search online for "ocean aerial dolphin" and get an URL:
url = "https://s3.envato.com/files/239350427/preview.jpg"

<div style="width:100%; margin-left:auto; margin-right:auto; margin-bottom:5px; margin-top:17px;">
<img src="https://s3.envato.com/files/239350427/preview.jpg" alt="IMAGE" /><br>
</div>

In [20]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

# Now there is a trained endpoint that can be used to make a prediction
dest_predictor = CustomVisionPredictionClient(dest_prediction_key, endpoint=ENDPOINT)

# Open the sample image and get back the prediction results.
results = dest_predictor.predict_image_url(dest_Project.id, iteration_id=iteration.id, url=url)

In [21]:
# Display the results.
for prediction in results.predictions:
    if(prediction.probability > 0.3): # only show the prediction with more than 30% threshold probability
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Object: 93.79%
	Ocean: 46.26%
